### Lendo os dados

In [1]:
import pandas as pd

df_raw = pd.read_csv("https://raw.githubusercontent.com/FelipeMMMendes/Machine-Learning-Churn-Project/main/Data/train_churn.csv", delimiter=";")

### Verificando os dados

In [2]:
df_raw.head()

,Unnamed: 0,genero,idoso,parceiro,dependentes,tempoDeServico,ServicoTelefone,MultiLinhas,ServicoInternet,ServicoSegurancaCyber,...,StreamingTV,StreamingFilmes,Contrato,BillingDigital,MetodoPagamento,FaturaMensal,FaturaTotal,NumTickets,NumTicketsTecnico,Churn
0,6027,Masculino,0.0,Sim,Sim,35.0,Sim,Sim,Não,Sem serviço de internet,...,Sem serviço de internet,Sem serviço de internet,2 Anos,Não,Transferência Bancária (Automática),25.45,809.25,0.0,0.0,0.0
1,1961,Feminino,0.0,Não,Não,42.0,Sim,Sim,DSL,NaN,...,Não,Não,1 Ano,Sim,Transferência Bancária (Automática),60.15,2421.6,0.0,0.0,0.0
2,5308,Feminino,0.0,Não,Não,69.0,Sim,Não,DSL,Sim,...,Sim,Sim,2 Anos,Sim,Cartão de Crédito (Automático),82.45,5555.3,0.0,0.0,0.0
3,3587,Masculino,1.0,Não,Não,9.0,Sim,NaN,Fibra óptica,Não,...,Sim,Sim,Mensal,Sim,Cartão de Crédito (Automático),99.45,919.4,0.0,0.0,1.0
4,576,Masculino,0.0,Sim,Sim,31.0,Sim,Sim,Fibra óptica,Sim,...,Sim,Não,1 Ano,Não,Transferência Bancária (Automática),98.05,3082.1,NaN,0.0,0.0


In [3]:
#deletando coluna Unnamed: 0

#vamos agora diferenciar, o df_raw eh o df cru, sem tratamento, eh no df em que faremos a limpeza dos dados
df = df_raw

df.drop("Unnamed: 0",axis=1,inplace=True)

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 22 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   genero                 4759 non-null   object 
 1   idoso                  4735 non-null   float64
 2   parceiro               4762 non-null   object 
 3   dependentes            4766 non-null   object 
 4   tempoDeServico         4747 non-null   float64
 5   ServicoTelefone        4789 non-null   object 
 6   MultiLinhas            4745 non-null   object 
 7   ServicoInternet        4771 non-null   object 
 8   ServicoSegurancaCyber  4778 non-null   object 
 9   ServicoBackup          4737 non-null   object 
 10  SeguroDispositivos     4750 non-null   object 
 11  ServicoSuporteTecnico  4767 non-null   object 
 12  StreamingTV            4751 non-null   object 
 13  StreamingFilmes        4775 non-null   object 
 14  Contrato               4738 non-null   object 
 15  Bill

In [5]:
#temos que fazer algumas normalizações dos dados, têm colunas que podem ser transformadas em números binários
#coluna genero
#Masculino vai passar a ser 1
#Feminino vai passar a ser 0
df['genero'] = df['genero'].map({'Masculino': 1, 'Feminino': 0})

In [6]:
#coluna parceiro
#Se tem parceiro (SIM) vai passar a ser 1
#Se não tem parceiro (NÃO) vai passar a ser 0
df['parceiro'] = df['parceiro'].map({'Sim': 1, 'Não': 0})

In [7]:
#coluna dependentes
#Se tem dependentes (SIM) vai passar a ser 1
#Se não tem dependentes (NÃO) vai passar a ser 0
df['dependentes'] = df['dependentes'].map({'Sim': 1, 'Não': 0})

In [11]:
#coluna BillingDigital
#Se tem billing digital (SIM) vai passar a ser 1
#Se não tem billing digital (NÃO) vai passar a ser 0
df['BillingDigital'] = df['BillingDigital'].map({'Sim': 1, 'Não': 0})

In [12]:
df.describe()

,genero,idoso,parceiro,dependentes,tempoDeServico,BillingDigital,FaturaMensal,NumTickets,NumTicketsTecnico,Churn
count,4759.00000,4735.000000,4762.000000,4766.000000,4747.000000,4762.000000,4726.000000,4731.000000,4745.000000,4736.000000
mean,0.50851,0.163464,0.487400,0.304029,32.158205,0.591978,64.801947,0.503488,0.411170,0.263091
std,0.49998,0.369827,0.499894,0.460043,29.592578,0.491519,35.930051,1.260113,1.230994,0.440358
min,0.00000,0.000000,0.000000,0.000000,-112.000000,0.000000,-127.000000,0.000000,0.000000,0.000000
25%,0.00000,0.000000,0.000000,0.000000,8.000000,0.000000,34.700000,0.000000,0.000000,0.000000
50%,1.00000,0.000000,0.000000,0.000000,29.000000,1.000000,70.400000,0.000000,0.000000,0.000000
75%,1.00000,0.000000,1.000000,1.000000,56.000000,1.000000,90.400000,0.000000,0.000000,1.000000
max,1.00000,1.000000,1.000000,1.000000,190.000000,1.000000,247.000000,5.000000,8.000000,1.000000


In [9]:
#separação em variáveis alvo e auxiliares
Y = df['Churn']
X = df.drop('Churn',axis=1)

In [10]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X,Y,test_size=0.2,random_state=42)

### Modelo CatBoostClassifier

In [27]:
#fazendo os imports necessarios
from catboost import CatBoostClassifier

model_catboost = CatBoostClassifier(iterations=1000, learning_rate=0.1)


### Modelo Rede Neural

### Modelo Árvore de Decisão

### Modelo Regressão Logística